# HW 2 - Naive Bayes in Hadoop MR
__`MIDS w261: Machine Learning at Scale | UC Berkeley School of Information | Fall 2018`__

In the live sessions for week 2 and week 3 you got some practice designing and debugging Hadoop Streaming jobs. In this homework we'll use Hadoop MapReduce to implement your first parallelized machine learning algorithm: Naive Bayes. As you develop your implementation you'll test it on a small dataset that matches the 'Chinese Example' in the _Manning, Raghavan and Shutze_ reading for Week 2. For the main task in this assignment you'll be working with a small subset of the Enron Spam/Ham Corpus. By the end of this assignment you should be able to:
* __... describe__ the Naive Bayes algorithm including both training and inference.
* __... perform__ EDA on a corpus using Hadoop MR.
* __... implement__ parallelized Naive Bayes.
* __... constrast__ partial, unordered and total order sort and their implementations in Hadoop Streaming.
* __... explain__ how smoothing affects the bias and variance of a Multinomial Naive Bayes model.

As always, your work will be graded both on the correctness of your output and on the clarity and design of your code. __Please refer to the `README` for homework submission instructions.__ 

## Notebook Setup
Before starting, run the following cells to confirm your setup.

In [1]:
# imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
# global vars (paths) - ADJUST AS NEEDED
JAR_FILE = "/usr/lib/hadoop-mapreduce/hadoop-streaming.jar"
HDFS_DIR = "/user/root/HW2/"
HOME_DIR = "/Users/ls_enel/Documents/MIDS/w261/main/Assigments/HW2" # FILL IN HERE eg. /media/notebooks/Assignments/HW2

In [3]:
# save path for use in Hadoop jobs (-cmdenv PATH={PATH})
from os import environ
PATH  = environ['PATH']

In [4]:
# data path
ENRON = "data/enronemail_1h.txt"

In [5]:
# make the HDFS directory if it doesn't already exist
!hdfs dfs -ls 
!hdfs dfs -mkdir {HDFS_DIR}

# Question 1: Hadoop MapReduce Key Takeaways.  

This assignment will be the only one in which you use Hadoop Streaming to implement a distributed algorithm. The key reason we continue to teach Hadoop streaming is because of the way it forces the programmer to think carefully about what is happening under the hood when you parallelize a calculation. This question will briefly highlight some of the most important concepts that you need to understand about Hadoop Streaming and MapReduce before we move on to Spark next week.   

### Q1 Tasks:

* __a) short response:__ What "programming paradigm" is Hadoop MapReduce based on? What are the main ideas of this programming paradigm and how does MapReduce exemplify these ideas?

* __b) short response:__ What is the Hadoop Shuffle? When does it happen? Why is it potentially costly? Describe one specific thing we can we do to mitigate the cost associated with this stage of our Hadoop Streaming jobs.

* __c) short response:__ In Hadoop Streaming why do the input and output record format of a combiner script have to be the same? [__`HINT`__ _what level of combining does the framework guarantee? what is the relationship between the record format your mapper emits and the format your reducer expects to receive?_]

* __d) short response:__ To what extent can you control the level of parallelization of your Hadoop Streaming jobs? Please be specific.

* __e) short response:__ What change in the kind of computing resources available prompted the creation of parallel computation frameworks like Hadoop? 

### Q1 Student Answers:

> __a)__ The divide and conequer programming paradigm of MapReduce is very reliant on stateless functional programming design. A divide-and-conquer strategy divides large problems into chunks of key-value records that can be processed by individual compute nodes. Ojbects that are stored in memory can't be broken down across multiple nodes.

> __b)__ The process by which the system performs the sort—and transfers the map outputs to the reducers as inputs. The major steps include: 1. partition, 2. sort, and 3. combine (in memory if possible or on disk). kevin - shuffle is important across the whole class. controlling when and how the shufffle occurs is very important.

> __c)__ In Hadoop, there is no guarantee that the combiner gets used so the mapper outputs must be formatted such that they CAN be used directly by the reducer. 

> __d)__ In Hadoop, we can explicitly call out how many parallel nodes we use - both on the mapper side and reducer side. The asynch videos recommended around 10–100 maps/node. The number of reducers should be designed such that:

- A multiple of the number of blocks
- A task time between 5 and 15 minutes
- Creating the fewest files possible

> __e)__ The availability of relatively cheap memory lead to frameworks like Hadoop where data/code was sent to memory, as memory is the most efficient place to get tasks done   

# Question 2: MapReduce Design Patterns.  

In the last two live sessions and in your readings from Lin & Dyer you encountered a number of techniques for manipulating the logistics of a MapReduce implementation to ensure that the right information is available at the right time and location. In this question we'll review a few of the key techniques you learned.   

### Q2 Tasks:

* __a) short response:__ What are counters (in the context of Hadoop Streaming)? How are they useful? What kinds of counters does Hadoop provide for you? How do you create your own custom counter?

* __b) short response:__ What are composite keys? How are they useful? How are they related to the idea of custom partitioning?

* __c) short response:__ What is the order inversion pattern? What problem does it help solve? How do we implement it? 

### Q2 Student Answers:

> __a)__  Hadoop Counters measure the number of operations that occur within the map-reduce job and are useful for gathering statistics about the map-reduce job for quality control and are also useful for problem diagnosis. Counters are a single variable that can be shared across multiple variables. Hadoop task counter collects information (e.g. number of records read and written) about tasks during its execution and are maintained by each task attempt and periodically sent to the application master to be globally aggregated. Information that the counters help capture include: number of bytes read and written by the file system, number of bytes read by map tasks, number of bytes written by map tasks to reduce tasks, the job-level statistics (i.e. not values that change while a task is running). To create a custom counter, we can use the code 'sys.stderr.write("reporter:counter: [function...])'

> __b)__ Composite keys are a way to tell Hadoop to treat some combination of variables as keys to partition on, allowing for multiple reducers to be used efficiently.  All lines with the same partition key are guaranteed to be processed by the same reducer.

> __c)__ order inverstion pattern is a technique by which we sort our systems outputs (e.g. print statement in a python script when called by hadoop) such that information that needs to be loaded logically is read first (e.g. when a reducer receives and output from mapper(s)). This helps prevent logical fallacies in our code like divide by zero errors. We implement this with the prepending of special characters to strings like "*" so that the information we want printed first is printed first when we sort the outputs.



# Question 3: Understanding Total Order Sort

The key challenge in distributed computing is to break a problem into a set of sub-problems that can be performed without communicating with each other. Ideally, we should be able to define an arbirtary number of splits and still get the right result, but that is not always possible. Parallelization becomes particularly challenging when we need to make comparisons between records, for example when sorting. Total Order Sort allows us to order large datasets in a way that enables efficient retrieval of results. Before beginning this assignment, make sure you have read and understand the [Total Order Sort Notebook](https://github.com/UCB-w261/main/tree/master/HelpfulResources/TotalSortGuide/_total-sort-guide-spark2.01-JAN27-2017.ipynb). You can skip the first two MRJob sections, but the rest of section III and all of section IV are **very** important (and apply to Hadoop Streaming) so make sure to read them closely. Feel free to read the Spark sections as well but you won't be responsible for that material until later in the course. To verify your understanding, answer the following questions.

### Q3 Tasks:

* __a) short response:__ What is the difference between a partial sort, an unordered total sort, and a total order sort? From the programmer's perspective, what does total order sort allow us to do that we can't with unordered total? Why is this important with large datasets?

* __b) short response:__ Which phase of a MapReduce job is leveraged to implement Total Order Sort? Which default behaviors must be changed. Why must they be changed?

* __c) short response:__ Describe in words how to configure a Hadoop Streaming job for the custom sorting and partitioning that is required for Total Order Sort.  

* __d) short response:__ Explain why we need to use an inverse hash code function.

* __e) short response:__ Where does this function need to be located so that a Total Order Sort can be performed?

### Q3 Student Answers:

> __a)__ 
- Partial sort - records are only ordered relative to the other keys that happen to be processed by the same reducer
- Unordered total sort - partition files themselves were out of order, even though the data is sorted within their paritions and also that the paritioning grouped records with similar value ranges together. This is because Hadoop's hash function on one's custom partition keys results in an ordering that may not match the human readible string or integer numbering that the programmer might have intended. This is particularly a problem with larger datasets where hash functions are incredible beneficial.
- Total Order Sort - sort the entire data set from highest to lowest.   

> __b)__  In the mapper is where we assign a custom partition key, as opposed to Hadoop's default, via alternative Hashing for TOS. This allows us to sort the parition keys in the order that Hadoop will

> __c)__ In the mapper, we will read the partition file and define specific cut points to assign letters for partition keys via a custom hash function that aligns with Hadoop. The reducer simply removes these partition keys. 

> __d)__ To preserve partition key ordering we use an inverse hash code function to return a partition key. The inputs to this function are the desired partition index and total number of partitions. That way this key will hash to the correct partition index via Hadoop.  

> __e)__ The inverse hashCode function is to used in the mapper via a lookup table.  


# About the Data
For the main task in this portion of the homework you will train a classifier to determine whether an email represents spam or not. You will train your Naive Bayes model on a 100 record subset of the Enron Spam/Ham corpus available in the HW2 data directory (__`HW2/data/enronemail_1h.txt`__).

__Source:__   
The original data included about 93,000 emails which were made public after the company's collapse. There have been a number raw and preprocessed versions of this corpus (including those available [here](http://www.aueb.gr/users/ion/data/enron-spam/index.html) and [here](http://www.aueb.gr/users/ion/publications.html)). The subset we will use is limited to emails from 6 Enron employees and a number of spam sources. It is part of [this data set](http://www.aueb.gr/users/ion/data/enron-spam/) which was created by researchers working on personlized Bayesian spam filters. Their original publication is [available here](http://www.aueb.gr/users/ion/docs/ceas2006_paper.pdf). __`IMPORTANT!`__ _For this homework please limit your analysis to the 100 email subset which we provide. No need to download or run your analysis on any of the original datasets, those links are merely provided as context._

__Preprocessing:__  
For their work, Metsis et al. (the authors) appeared to have pre-processed the data, not only collapsing all text to lower-case, but additionally separating "words" by spaces, where "words" unfortunately include punctuation. As a concrete example, the sentence:  
>  `Hey Jon, I hope you don't get lost out there this weekend!`  

... would have been reduced by Metsis et al. to the form:  
> `hey jon , i hope you don ' t get lost out there this weekend !` 

... so we have reverted the data back toward its original state, removing spaces so that our sample sentence would now look like:
> `hey jon, i hope you don't get lost out there this weekend!`  

Thus we have at least preserved contractions and other higher-order lexical forms. However, one must be aware that this reversion is not complete, and that some object (specifically web sites) will be ill-formatted, and that all text is still lower-cased.


__Format:__   
All messages are collated to a tab-delimited format:  

>    `ID \t SPAM \t SUBJECT \t CONTENT \n`  

where:  
>    `ID = string; unique message identifier`  
    `SPAM = binary; with 1 indicating a spam message`  
    `SUBJECT = string; title of the message`  
    `CONTENT = string; content of the message`   
    
Note that either of `SUBJECT` or `CONTENT` may be "NA", and that all tab (\t) and newline (\n) characters have been removed from both of the `SUBJECT` and `CONTENT` columns.  

In [6]:
!pwd

/media/notebooks/Assignments/HW2


In [7]:
# take a look at the first 100 characters of the first 5 records (RUN THIS CELL AS IS)
!head -n 5 /media/notebooks/Assignments/HW2/{ENRON} | cut -c-100

0001.1999-12-10.farmer	0	 christmas tree farm pictures	NA
0001.1999-12-10.kaminski	0	 re: rankings	 thank you.
0001.2000-01-17.beck	0	 leadership development pilot	" sally:  what timing, ask and you shall receiv
0001.2000-06-06.lokay	0	" key dates and impact of upcoming sap implementation over the next few week
0001.2001-02-07.kitchen	0	 key hr issues going forward	 a) year end reviews-report needs generating 


In [8]:
# see how many messages/lines are in the file 
#(this number may be off by 1 if the last line doesn't end with a newline)
!wc -l /media/notebooks/Assignments/HW2/{ENRON}

100 /media/notebooks/Assignments/HW2/data/enronemail_1h.txt


In [9]:
# make the HDFS directory if it doesn't already exist
!hdfs dfs -mkdir {HDFS_DIR}

mkdir: `/user/root/HW2': File exists


In [10]:
# load the data into HDFS (RUN THIS CELL AS IS)
!hdfs dfs -copyFromLocal /media/notebooks/Assignments/HW2/{ENRON} {HDFS_DIR}/enron.txt

In [11]:
!hdfs dfs -ls {HDFS_DIR}

Found 1 items
-rw-r--r--   1 root supergroup     204559 2021-09-22 03:44 /user/root/HW2/enron.txt


# Question 4:  Enron Ham/Spam EDA.
Before building our classifier, lets get aquainted with our data. In particular, we're interested in which words occur more in spam emails than in legitimate ("ham") emails. In this question you'll implement two Hadoop MapReduce jobs to count and sort word occurrences by document class. You'll also learn about two new Hadoop streaming parameters that will allow you to control how the records output from your mappers are partitioned for reducing on separate nodes. 

__`IMPORTANT NOTE:`__ For this question and all subsequent items, you should include both the subject and the body of the email in your analysis (i.e. concatetate them to get the 'text' of the document).

### Q4 Tasks:
* __a) code:__ Complete the missing components of the code in __`EnronEDA/mapper.py`__ and __`EnronEDA/reducer.py`__ to create a Hadoop  MapReduce job that counts how many times each word in the corpus occurs in an email for each class. Pay close attention to the data format specified in the docstrings of these scripts _-- there are a number of ways to accomplish this task, we've chosen this format to help illustrate a technique in `part e`_. Run the provided unit tests to confirm that your code works as expected, then run the provided Hadoop Streaming command to apply your analysis to the Enron data.


* __b) code + short response:__ How many times does the word "__assistance__" occur in each class? (`HINT:` Use a `grep` command to read from the results file you generated in '`a`' and then report the answer in the space provided.)


* __c) short response:__ Would it have been possible to add some sorting parameters to the Hadoop streaming command that would cause our `part a` results to be sorted by count? Explain why or why not. (`HINT:` This question demands an understanding of the sequence of the phases of MapReduce.)


* __d) code + short response:__ Write a second Hadoop MapReduce job to sort the output of `part a` first by class and then by count. Run your job and save the results to a local file. Then describe in words how you would go about printing the top 10 words in each class given this sorted output. (`HINT 1:` _remember that you can simply pass the `part a` output directory to the input field of this job; `HINT 2:` since this task is just reodering the records from `part a` we don't need to write a mapper or reducer, just use `/bin/cat` for both_)


* __e) code:__ A more efficient alternative to '`grep`-ing' for the top 10 words in each class would be to use the Hadoop framework to separate records from each class into its own partition so that we can just read the top lines in each. Rewrite your job from ` part d` to specify 2 reduce tasks and to tell Hadoop to partition based on the second field (which indicates spam/ham in our data). Your code should maintain the secondary sort -- that is each partition should list words from most to least frequent.

### Q4 Student Answers:
> __b)__ the word 'assistance' appeared once in the ham class and 11 times in the spam class.

> __c)__ Not without passing the outputs of the reducer to another map reduce job that sorts by column value.  

> __d)__ By partitioning the data by the class key and using two reducers, we can save each class's results to two seperate hadoop output files. Then we can loop through these files and use the head command to take a look at the top files for each class by count.  

In [12]:
# part a - do your work in the provided scripts then RUN THIS CELL AS IS
!chmod a+x EnronEDA/mapper.py
!chmod a+x EnronEDA/reducer.py

In [13]:
# part a - unit test EnronEDA/mapper.py (RUN THIS CELL AS IS)
!echo -e "d1	1	title	body\nd2	0	title	body" | EnronEDA/mapper.py

title	1	1
body	1	1
title	0	1
body	0	1


In [14]:
# part a - unit test EnronEDA/reducer.py (RUN THIS CELL AS IS)
!echo -e "one	1	1\none	0	1\none	0	1\ntwo	0	1" | EnronEDA/reducer.py

one	0	2
one	1	1
two	0	1
two	1	0


In [15]:
# part a - clear output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-output

rm: `/user/root/HW2//eda-output': No such file or directory


In [16]:
!hdfs dfs -rm -r {HDFS_DIR}/eda-output
# part a - Hadoop streaming job (RUN THIS CELL AS IS)
!hadoop jar {JAR_FILE} \
  -files EnronEDA/reducer.py,EnronEDA/mapper.py \
  -mapper mapper.py \
  -reducer reducer.py \
  -input {HDFS_DIR}/enron.txt \
  -output {HDFS_DIR}/eda-output \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH}

rm: `/user/root/HW2//eda-output': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob7576194625936759740.jar tmpDir=null
21/09/22 03:45:03 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/22 03:45:04 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/22 03:45:05 INFO mapred.FileInputFormat: Total input paths to process : 1
21/09/22 03:45:05 INFO mapreduce.JobSubmitter: number of splits:2
21/09/22 03:45:05 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1632272621573_0001
21/09/22 03:45:06 INFO impl.YarnClientImpl: Submitted application application_1632272621573_0001
21/09/22 03:45:07 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1632272621573_0001/
21/09/22 03:45:07 INFO mapreduce.Job: Running job: job_1632272621573_0001
21/09/22 03:45:20 INFO mapreduce.Job: Job job_1632272621573_0001 running in uber mode : false
2

In [17]:
# part a - retrieve results from HDFS & copy them into a local file (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/eda-output/part-0000* > EnronEDA/results.txt

In [18]:
# part b - write your grep command here
!grep -i "assistance" EnronEDA/results.txt

assistance	0	3
assistance	1	8


In [19]:
# part d - clear the output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-sort-output

rm: `/user/root/HW2//eda-sort-output': No such file or directory


In [20]:
# part d - write your Hadoop streaming job here
!hadoop jar {JAR_FILE} \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D stream.num.map.output.key.fields=3 \
  -D mapreduce.partition.keycomparator.options="-k2n -k3nr" \
  -files EnronEDA/mapper.py,EnronEDA/reducer.py \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -mapper /bin/cat\
  -reducer /bin/cat\
  -input {HDFS_DIR}/eda-output \
  -output {HDFS_DIR}/eda-sort-output \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob704150810451840245.jar tmpDir=null
21/09/22 03:45:48 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/22 03:45:48 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/22 03:45:49 INFO mapred.FileInputFormat: Total input paths to process : 2
21/09/22 03:45:50 INFO mapreduce.JobSubmitter: number of splits:2
21/09/22 03:45:50 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1632272621573_0002
21/09/22 03:45:50 INFO impl.YarnClientImpl: Submitted application application_1632272621573_0002
21/09/22 03:45:50 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1632272621573_0002/
21/09/22 03:45:50 INFO mapreduce.Job: Running job: job_1632272621573_0002
21/09/22 03:46:01 INFO mapreduce.Job: Job job_1632272621573_0002 running in uber mode : false
21/09/22 03:46:01 INFO mapreduce.Job:  map 0% reduce 0%
21/09/

In [21]:
!hdfs dfs -cat {HDFS_DIR}/eda-sort-output/part-00000 | head -n 10

to	0	399	
and	0	278	
hou	0	206	
in	0	182	
will	0	132	
enron	0	127	
with	0	86	
by	0	67	
are	0	62	
s	0	58	
cat: Unable to write to output stream.


In [22]:
# part e - clear the output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-sort-output

Deleted /user/root/HW2/eda-sort-output


In [23]:
#Input format into mapper:
#"d1	1	title	body\
#d2	0	title	body"

#Input format into the reducer:
#title	1	1
#body	1	1

!hdfs dfs -rm -r {HDFS_DIR}/eda-sort-output

!hadoop jar {JAR_FILE} \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D stream.num.map.output.key.fields=3 \
  -D mapreduce.partition.keycomparator.options="-k2,2r -k3,3nr" \
  -D mapreduce.partition.keypartitioner.options="-k2,2"  \
  -files EnronEDA/mapper.py,EnronEDA/reducer.py \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -mapper /bin/cat\
  -reducer /bin/cat\
  -input {HDFS_DIR}/eda-output \
  -output {HDFS_DIR}/eda-sort-output \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH}
#  -D mapreduce.partition.keycomparator.options="-k3,3nr" \


rm: `/user/root/HW2//eda-sort-output': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob6446228858571814200.jar tmpDir=null
21/09/22 03:46:32 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/22 03:46:33 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/22 03:46:34 INFO mapred.FileInputFormat: Total input paths to process : 2
21/09/22 03:46:34 INFO mapreduce.JobSubmitter: number of splits:2
21/09/22 03:46:34 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1632272621573_0003
21/09/22 03:46:34 INFO impl.YarnClientImpl: Submitted application application_1632272621573_0003
21/09/22 03:46:34 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1632272621573_0003/
21/09/22 03:46:34 INFO mapreduce.Job: Running job: job_1632272621573_0003
21/09/22 03:46:44 INFO mapreduce.Job: Job job_1632272621573_0003 running in uber mode : fa

In [24]:
# part e - view the top 10 records from each partition (RUN THIS CELL AS IS)
for idx in range(2):
    print(f"\n===== part-0000{idx}=====\n")
    !hdfs dfs -cat {HDFS_DIR}/eda-sort-output/part-0000{idx} | head


===== part-00000=====

the	0	550	
to	0	399	
ect	0	382	
and	0	278	
of	0	231	
hou	0	206	
a	0	196	
in	0	182	
for	0	170	
on	0	136	
cat: Unable to write to output stream.

===== part-00001=====

the	1	698	
to	1	566	
and	1	393	
your	1	357	
a	1	347	
you	1	345	
of	1	336	
in	1	237	
for	1	205	
com	1	153	
cat: Unable to write to output stream.


__Expected output:__
<table>
<th>part-00000:</th>
<th>part-00001:</th>
<tr><td><pre>
the	0	549	
to	0	398	
ect	0	382	
and	0	278	
of	0	230	
hou	0	206	
a	0	196	
in	0	182	
for	0	170	
on	0	135
</pre></td>
<td><pre>
the	1	698	
to	1	566	
and	1	392	
your	1	357	
a	1	347	
you	1	345	
of	1	336	
in	1	236	
for	1	204	
com	1	153
</pre></td></tr>
</table>

# Question 5: Counters and Combiners.
Tuning the number of mappers & reducers is helpful to optimize very large distributed computations. Doing so successfully requires a thorough understanding of the data size at each stage of the job. As you learned in the week3 live session, counters are an invaluable resource for understanding this kind of detail. In this question, we will take the EDA performed in Question 4 as an opportunity to illustrate some related concepts.

### Q5 Tasks:
* __a) short response:__ Read the Hadoop output from your job in Question 4a to report how many records are emitted by the mappers and how many records are received be the reducers. In the context of word counting what does this number represent practically?

* __b) code:__ Note that we wrote the reducer in question 4a such that the input and output record format is identical. This makes it easy to use the same reducer script as a combiner. In the space provided below, write the Hadoop Streaming command to re-run your job from question 4a with this combining added.

* __c) short response__: Report the number of records emitted by your mappers in part b and the number of records received by your reducers. Compare your results here to what you saw in part a. Explain.

* __d) short response__: Describe a scenario where using a combiner would _NOT_ improve the efficiency of the shuffle stage. Explain. [__`BONUS:`__ how does increasing the number of mappers affect the usefulness of a combiner?]

### Q5 Student Answers:
> __a)__ 
- Map input records=100 - how many messages/lines are in the file
- Map output records=31490 - how many lines across all mappers (i.e. combo of class-word across all mappers) 
- Reduce input records=31490 - input lines into the reducers
- Reduce output records=10130 - output of the reducers (i.e. number of unique class-word combinations)
        
The map input records  whereas the the number of records is the number of the number of unique class-word configurations spread across the mappers. 

> __c)__ Type your answer here!
- Map input records=100
- Map output records=31490
- Combine input records=31490
- Combine output records=13096
- Reduce input records=13096
- Reduce output records=10130

The inputs/outputs of the mappers match as expected. The input into the reducer is smaller though indicating that the combiners successfully aggregated some of the mapper outputs before sending to the reducers.

> __d)__ If only one mapper and one reducer is used, then adding a combiner processes the data twice unnecessarily. Otherwise a scenario may not be useful if the data being processed is relatively small and the communication costs betweeen map-combiner-reducer doesn't offset that added costs. 

In [25]:
# part b - clear output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-output

Deleted /user/root/HW2/eda-output


In [26]:
# part b - write your Hadoop streaming job here
!hadoop jar {JAR_FILE} \
  -files EnronEDA/reducer.py,EnronEDA/mapper.py \
  -mapper mapper.py \
  -combiner reducer.py \
  -reducer reducer.py \
  -input {HDFS_DIR}/enron.txt \
  -output {HDFS_DIR}/eda-output \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH}

#Question 4A, no combiner
# !hadoop jar {JAR_FILE} \
#   -files EnronEDA/reducer.py,EnronEDA/mapper.py \
#   -mapper mapper.py \
#   -reducer reducer.py \
#   -input {HDFS_DIR}/enron.txt \
#   -output {HDFS_DIR}/eda-output \
#   -numReduceTasks 2 \
#   -cmdenv PATH={PATH}


packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob2451827032707840122.jar tmpDir=null
21/09/22 03:47:16 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/22 03:47:16 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/22 03:47:17 INFO mapred.FileInputFormat: Total input paths to process : 1
21/09/22 03:47:17 INFO mapreduce.JobSubmitter: number of splits:2
21/09/22 03:47:18 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1632272621573_0004
21/09/22 03:47:18 INFO impl.YarnClientImpl: Submitted application application_1632272621573_0004
21/09/22 03:47:18 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1632272621573_0004/
21/09/22 03:47:18 INFO mapreduce.Job: Running job: job_1632272621573_0004
21/09/22 03:47:28 INFO mapreduce.Job: Job job_1632272621573_0004 running in uber mode : false
21/09/22 03:47:28 INFO mapreduce.Job:  map 0% reduce 0%
21/09

In [27]:
# part b - retrieve results from HDFS
# !hdfs dfs -cat {HDFS_DIR}/eda-output/part-0000* | head
!hdfs dfs -cat {HDFS_DIR}/eda-output/part-0000* > EnronEDA/results_q5.txt

# Question 6: Document Classification Task Overview.
The week 2 assigned reading from Chapter 13 of _Introduction to Information Retrieval_ by Manning, Raghavan and Schutze provides a thorough introduction to the document classification task and the math behind Naive Bayes. In this question we'll use the example from Table 13.1 (reproduced below) to 'train' an unsmoothed Multinomial Naive Bayes model and classify a test document by hand.

<table>
<th>DocID</th>
<th>Class</th>
<th>Subject</th>
<th>Body</th>
<tr><td>Doc1</td><td>1</td><td></td><td>Chinese Beijing Chinese</td></tr>
<tr><td>Doc2</td><td>1</td><td></td><td>Chinese Chinese Shanghai</td></tr>
<tr><td>Doc3</td><td>1</td><td></td><td>Chinese Macao</td></tr>
<tr><td>Doc4</td><td>0</td><td></td><td>Tokyo Japan Chinese</td></tr>
</table>

### Q6 Tasks:
* __a) short response:__ Equation 13.3 in Manning, Raghavan and Shutze shows how a Multinomial Naive Bayes model classifies a document. It predicts the class, $c$, for which the estimated conditional probability of the class given the document's contents,  $\hat{P}(c|d)$, is greatest. In this equation what two pieces of information are required to calculate  $\hat{P}(c|d)$? Your answer should include both mathematical notatation and verbal explanation.


* __b) short response:__ The Enron data includes two classes of documents: `spam` and `ham` (they're actually labeled `1` and `0`). In plain English, explain what  $\hat{P}(c)$ and   $\hat{P}(t_{k} | c)$ mean in the context of this data. How will we would estimate these values from a training corpus? How many passes over the data would we need to make to retrieve this information for all classes and all words?


* __c) hand calculations:__ Above we've reproduced the document classification example from the textbook (we added an empty subject field to mimic the Enron data format). Remember that the classes in this "Chinese Example" are `1` (about China) and `0` (not about China). Calculate the class priors and the conditional probabilities for an __unsmoothed__ Multinomial Naive Bayes model trained on this data. Show the calculations that lead to your result using markdown and $\LaTeX$ in the space provided or by embedding an image of your hand written work. [`NOTE:` _Your results should NOT match those in the text -- they are training a model with +1 smoothing you are training a model without smoothing_]


* __d) hand calculations:__ Use the model you trained to classify the following test document: `Chinese Chinese Chinese Tokyo Japan`. Show the calculations that lead to your result using markdown and   $\LaTeX$ in the space provided or by embedding an image of your hand written work.


* __e) short response:__ Compare the classification you get from this unsmoothed model in `d`/`e` to the results in the textbook's "Example 1" which reflects a model with Laplace plus 1 smoothing. How does smoothing affect our inference?

### Q6 Student Answers:
> __a)__ To calculate the probability of a class (c) given a document (d), you need to know the prior probability of a document occurring in class c (i.e. $\hat{P}(c)$) as well as the conditional probability of token ($t_k$) occurring in a document of class c. These values will be estimates given they can only be learned over the training sample and not the full population. The probability of a document d being in class c is approximated by: 
$\hat{P}(c|d) = \hat{P}(c) * (\hat{P}(t_1|c) * ... \hat{P}(t_{n_d}|c))$ where $n_d$ is the number of such tokens in d.

> __b)__ $\hat{P}(c)$ is the prior probability of a document occuring in a class, that is without learning anything about the contents of the document, should our default guess be that it is spam or ham. $\hat{P}(t_{k} | c)$ is it conditional probability of a given token $t_k$, that is what is the probability of witnesssing this token in a document that is a spam or ham document...

> __c)__ 

$ClassPriors_{ham}$: #HamDocs/TotalDocs = $\frac{0 + 0 + 0 + 1}{1 + 1 + 1 + 1} = 25\%$ 

$ClassPriors_{spam}$: #SpamDocs/TotalDocs = $\frac{1 + 1 + 1 + 0}{1 + 1 + 1 + 1} = 75%$

$P(word | ham)$: #word occurence in ham docs/total words in ham docs. Reverse for spam

- $P(Chinese | ham)$: 1/3,  $P(Chinese | spam)$: 5/8
- $P(Bejing | ham)$: 0/3,  $P(Bejing | spam)$: 1/8
- $P(Shanghai | ham)$: 0/3,  $P(Shanghai | spam)$: 1/8
- $P(Macao | ham)$: 0/3,  $P(Macao | spam)$: 1/8
- $P(Tokyo | ham)$: 1/3,  $P(Tokyo | spam)$: 0/8
- $P(Japan | ham)$: 1/3,  $P(Japan | spam)$: 0/8

> __d)__ doc: Chinese Chinese Chinese Tokyo Japan

$P(doc) = spam: ClassPriors_{spam}*P(Chinese | spam)^3*P(Tokyo | spam)*P(Japan | spam) = 0.75*(5/8)^3*0*0 = 0
$P(doc) = ham: ClassPriors_{ham}*P(Chinese | ham)^3*P(Tokyo | spam)*P(Japan | ham) = 0.25*(1/3)^3*1/3*1/3 = 0.0010288

the doc is ham (i.e. not chinese)

> __e)__ With smoothing, the document is classified as spam (i.e. chinese). Smoothing removes the zero probability multiplier from not having Tokyo or Japan in any of the spam documents. 


In [28]:
# part d/e - if you didn't write out your calcuations above, embed a picture of them here:
#TODO uncomment
# from IPython.display import Image
# Image(filename="path-to-hand-calulations-image.png")

# Question 7: Naive Bayes Inference.
In the next two questions you'll write code to parallelize the Naive Bayes calculations that you performed above. We'll do this in two phases: one MapReduce job to perform training and a second MapReduce to perform inference. While in practice we'd need to train a model before we can use it to classify documents, for learning purposes we're going to develop our code in the opposite order. By first focusing on the pieces of information/format we need to perform the classification (inference) task you should find it easier to develop a solid implementation for training phase when you get to question 8 below. In both of these questions we'll continue to use the Chinese example corpus from the textbook to help us test our MapReduce code as we develop it. Below we've reproduced the corpus, test set and model in text format that matches the Enron data.

### Q7 Tasks:
* __a) short response:__ run the provided cells to create the example files and load them in to HDFS. Then take a closer look at __`NBmodel.txt`__. This text file represents a Naive Bayes model trained (with Laplace +1 smoothing) on the example corpus. What are the 'keys' and 'values' in this file? Which record means something slightly different than the rest? The value field of each record includes two numbers which will be helpful for debugging but which we don't actually need to perform inference -- what are they? [`HINT`: _This file represents the model from Example 13.1 in the textbook, if you're having trouble getting oriented try comparing our file to the numbers in that example._]


* __b) short response:__ When performing Naive Bayes in practice instead of multiplying the probabilities (as in equation 13.3) we add their logs (as in equation 13.4). Why do we choose to work with log probabilities? If we had an unsmoothed model, what potential error could arise from this transformation?


* __c) short response:__ Documents 6 and 8 in the test set include a word that did not appear in the training corpus (and as a result does not appear in the model). What should we do at inference time when we need a class conditional probability for this word?


* __d) short response:__ The goal of our MapReduce job is to stream over the test set and classify each document by peforming the calculation from equation 13.4. To do this we'll load the model file (which contains the probabilities for equation 13.4) into memory on the nodes where we do our mapping. This is called an in-memory join. Does loading a model 'state' like this depart from the functional programming principles? Explain why or why not. From a scability perspective when would this kind of memory use be justified? when would it be unwise?


* __e) code:__ Complete the code in __`NaiveBayes/classify_mapper.py`__. Read the docstring carefully to understand how this script should work and the format it should return. Run the provided unit tests to confirm that your script works as expected then write a Hadoop streaming job to classify the Chinese example test set. [`HINT 1:` _you shouldn't need a reducer for this one._ `HINT 2:` _Don't forget to add the model file to the_ `-files` _parameter in your Hadoop streaming job so that it gets shipped to the mapper nodes where it will be accessed by your script._]


* __f) short response:__ In our test example and in the Enron data set we have fairly short documents. Since these fit fine in memory on a mapper node we didn't need a reducer and could just do all of our calculations in the mapper. However with much longer documents (eg. books) we might want a higher level of parallelization -- for example we might want to process parts of a document on different nodes. In this hypothetical scenario how would our algorithm design change? What could the mappers still do? What key-value structure would they emit? What would the reducers have to do as a last step?

### Q7 Student Answers:
> __a)__ The keys are the token labels and the values are the conditional probabilities of those tokens given that they belong to chinese class. The one entry that differs is the ClassPriors line, which gives the prior probabilities of the documents in the training set given that they are in the chinese class

> __b)__ We work with log probabilites since the properties of logs allow us to add the probabilities instead of multiplying them, which could quickly result in a floating point underflow error in digital computations if we were to multiply non log transformed probabilities. If we had an unsmoothed model, then a word appears in the testing set but not in the training set will result in the P(word | C=chinese) and P(word | not chinese) = 0. This breaks our probability multiplication equation. Thus we apply smoothing techniques to account for this potential mathematical error. 

> __c)__ Two approaches could be: 1. assign it a value of 1 which means the probability that the word is from the chinese class and not chinese class are equal or 2. we can apply smoothing, which likely will require two MapReduce jobs - one to calculate the vocabulary size using one or more reducers and a second job to calculate the conditonal probabilities

> __d)__ I think does not practically violate our functional programming princicples as the state of the model that we load into the memory of the node does not chance as we do our mapping. Even though the model has a state, it is practically like loading a function into our mappers. Compared to a large corpous, which may not fit in the memory of one mapper, we likely will be able to fit a naive bayes classifer into memory.  

> __e)__ Complete the coding portion of this question before answering 'f'.

> __f)__ As the size of our corpous increases, we will need to split the calculation across multiple nodes. Thus for classification, we will need to move the summation from the mapper to the reducer to predict the class.  





Run these cells to create the example corpus and model.

In [29]:
%%writefile NaiveBayes/chineseTrain.txt
D1	1		Chinese Beijing Chinese
D2	1		Chinese Chinese Shanghai
D3	1		Chinese Macao
D4	0		Tokyo Japan Chinese

Overwriting NaiveBayes/chineseTrain.txt


In [30]:
%%writefile NaiveBayes/chineseTest.txt
D5	1		Chinese Chinese Chinese Tokyo Japan
D6	1		Beijing Shanghai Trade
D7	0		Japan Macao Tokyo
D8	0		Tokyo Japan Trade

Overwriting NaiveBayes/chineseTest.txt


In [31]:
%%writefile NBmodel.txt
beijing	0.0,1.0,0.111111111111,0.142857142857
chinese	1.0,5.0,0.222222222222,0.428571428571
tokyo	1.0,0.0,0.222222222222,0.0714285714286
shanghai	0.0,1.0,0.111111111111,0.142857142857
ClassPriors	1.0,3.0,0.25,0.75
japan	1.0,0.0,0.222222222222,0.0714285714286
macao	0.0,1.0,0.111111111111,0.142857142857

Overwriting NBmodel.txt


In [32]:
# load the data files into HDFS
!hdfs dfs -copyFromLocal NaiveBayes/chineseTrain.txt {HDFS_DIR}
!hdfs dfs -copyFromLocal NaiveBayes/chineseTest.txt {HDFS_DIR}

Your work for `part e` starts here:

In [33]:
# part e - do your work in NaiveBayes/classify_mapper.py first, then run this cell.
!chmod a+x NaiveBayes/classify_mapper.py

In [88]:
!cat NaiveBayes/classify_mapper.py

#!/usr/bin/env python
"""
Mapper for Naive Bayes Inference.
INPUT:
    ID \t true_class \t subject \t body \n
OUTPUT:
    ID \t true_class \t logP(ham|doc) \t logP(spam|doc) \t predicted_class
SUPPLEMENTAL FILE: 
    This script requires a trained Naive Bayes model stored 
    as NBmodel.txt in the current directory. The model should 
    be a tab separated file whose records look like:
        WORD \t ham_count,spam_count,P(word|ham),P(word|spam)
        
Instructions:
    We have loaded the supplemental file and taken the log of 
    each conditional probability in the model. We also provide
    the code to tokenize the input lines for you. Keep in mind 
    that each 'line' of this file represents a unique document 
    that we wish to classify. Fill in the missing code to get
    the probability of each class given the words in the document.
    Remember that you will need to handle the case where you
    encounter a word that is not represented in the model.
"""
import os
import r

In [35]:
# part e - unit test NaiveBayes/classify_mapper.py (RUN THIS CELL AS IS)
!cat NaiveBayes/chineseTest.txt | NaiveBayes/classify_mapper.py | column -t

d5  1  -8.90668134500626   -8.10769031284611   1
d6  1  -5.780743515794329  -4.179502370564408  1
d7  0  -6.591673732011658  -7.511706880737811  0
d8  0  -4.394449154674438  -5.565796731681498  0


In [36]:
# part e - clear the output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/chinese-output

rm: `/user/root/HW2//chinese-output': No such file or directory


In [37]:
!hdfs dfs -rm -r {HDFS_DIR}/chinese-output
# part e - write your Hadooop streaming job here

!hadoop jar {JAR_FILE} \
  -files NaiveBayes/classify_mapper.py,NBmodel.txt \
  -mapper classify_mapper.py \
  -input {HDFS_DIR}/chineseTest.txt \
  -output {HDFS_DIR}/chinese-output \
  -cmdenv PATH={PATH}



rm: `/user/root/HW2//chinese-output': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob7702695767764060217.jar tmpDir=null
21/09/22 03:48:16 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/22 03:48:16 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/22 03:48:17 INFO mapred.FileInputFormat: Total input paths to process : 1
21/09/22 03:48:17 INFO mapreduce.JobSubmitter: number of splits:2
21/09/22 03:48:18 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1632272621573_0005
21/09/22 03:48:18 INFO impl.YarnClientImpl: Submitted application application_1632272621573_0005
21/09/22 03:48:18 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1632272621573_0005/
21/09/22 03:48:18 INFO mapreduce.Job: Running job: job_1632272621573_0005
21/09/22 03:48:26 INFO mapreduce.Job: Job job_1632272621573_0005 running in uber mode : fal

In [38]:
# part e - retrieve test set results from HDFS (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/chinese-output/part-000* > NaiveBayes/chineseResults.txt

In [39]:
# part e - take a look (RUN THIS CELL AS IS)
!cat NaiveBayes/chineseResults.txt | column -t

d5  1  -8.90668134500626   -8.10769031284611   1
d6  1  -5.780743515794329  -4.179502370564408  1
d7  0  -6.591673732011658  -7.511706880737811  0
d8  0  -4.394449154674438  -5.565796731681498  0


<table>
<th> Expected output for the test set:</th>
<tr align=Left><td><pre>
d5	1	-8.90668134	-8.10769031	1
d6	1	-5.78074351	-4.17950237	1
d7	0	-6.59167373	-7.51170688	0
d8	0	-4.39444915	-5.56579673	0
</pre></td><tr>
</table>

# Question 8: Naive Bayes Training.
In Question 7 we used a model that we had trained by hand. Next we'll develop the code to do that same training in parallel, making it suitable for use with larger corpora (like the Enron emails). The end result of the MapReduce job you write in this question should be a model text file that looks just like the example (`NBmodel.txt`) that we created by hand above.

To refresh your memory about the training process take a look at  `6a` and `6b` where you described the pieces of information you'll need to collect in order to encode a Multinomial Naive Bayes model. We now want to retrieve those pieces of information while streaming over a corpus. The bulk of the task will be very similar to the word counting excercises you've already done but you may want to consider a slightly different key-value record structure to efficiently tally counts for each class. 

The most challenging (interesting?) design question will be how to retrieve the totals (# of documents and # of words in documents for each class). Of course, counting these numbers is easy. The hard part is the timing: you'll need to make sure you have the counts totalled up _before_ you start estimating the class conditional probabilities for each word. It would be best (i.e. most scalable) if we could find a way to do this tallying without storing the whole vocabulary in memory... Use an appropriate MapReduce design pattern to implement this efficiently! 


### Q8 Tasks:
* __a) make a plan:__  Fill in the docstrings for __`NaiveBayes/train_mapper.py`__ and __`NaiveBayes/train_reducer.py`__ to appropriately reflect the format that each script will input/output. [`HINT:` _the input files_ (`enronemail_1h.txt` & `chineseTrain.txt`) _have a prespecified format and your output file should match_ `NBmodel.txt` _so you really only have to decide on an internal format for Hadoop_].


* __b) implement it:__ Complete the code in __`NaiveBayes/train_mapper.py`__ and __`NaiveBayes/train_reducer.py`__ so that together they train a Multinomial Naive Bayes model __with no smoothing__. Make sure your end result is formatted correctly (see note above). Test your scripts independently and together (using `chineseTrain.txt` or test input of your own devising). When you are satisfied with your Python code design and run a Hadoop streaming command to run your job in parallel on the __chineseTrain.txt__. Confirm that your trained model matches your hand calculations from Question 6.


* __c) short response:__ We saw in Question 6 that adding Laplace smoothing (where the smoothing parameter $k=1$) makes our classifications less sensitve to rare words. However implementing this technique requires access to one additional piece of information that we had not previously used in our Naive Bayes training. What is that extra piece of information? [`HINT:` see equation 13.7 in Manning, Raghavan and Schutze].


* __d) short response:__ There are a couple of approaches that we could take to handle the extra piece of information you identified in `c`: 1) if we knew this extra information beforehand, we could provide it to our reducer as a configurable parameter for the vocab size dynamically (_where would we get it in the first place?_). Or 2) we could compute it in the reducer without storing any bulky information in memory but then we'd need some postprocessing or a second MapReduce job to complete the calculation (_why?_). Breifly explain what is non-ideal about each of these options. 


* __e) code + short response:__ Choose one of the 2 options above. State your choice & reasoning in the space below then use that strategy to complete the code in __`NaiveBayes/train_reducer_smooth.py`__. Test this alternate reducer then write and run a Hadoop streaming job to train an MNB model with smoothing on the Chinese example. Your results should match the model that we provided for you above (and the calculations in the textbook example). __IMPORTANT NOTE:__ For full credit on this question, your code must work with multiple reducers. 

    - [`HINT:` You will need to implement custom partitioning - [Total Order Sort Notebook](https://github.com/UCB-w261/main/tree/master/HelpfulResources/TotalSortGuide/_total-sort-guide-spark2.01-JAN27-2017.ipynb)] 

    - [`HINT:` Don't start from scratch with this one -- you can just copy over your reducer code from part `b` and make the needed modifications]. 



__IMPORTANT NOTE:__ For full credit on this question, your code must work with multiple reducers. [`HINT:`_You will need to implement custom partitioning - [Total Order Sort Notebook](https://github.com/UCB-w261/main/tree/master/HelpfulResources/TotalSortGuide/_total-sort-guide-spark2.01-JAN27-2017.ipynb)]


### Q8 Student Answers:
> __ c)__ For laplace smoothing, we need information about the total size of the unique vocabulary for each reducer.

> __ d)__ If we knew the size of the vocavulary beforehand, we could hard code this as a parameter however the issue is this does not scale if the corpous is too large to preprocess to determine the vocabulary size. If we used two map-reduce jobs, we can calculate this in the hadoop job however this still requires that we use only one reduce job to determine the vocabulary size as well as the added computation time to run two map reduce jobs.

> __ e)__ I decided to use the multiple map reduce jobs so to better implement scalable solutions (as in general the single reduce job should be able to the size of the unique dictionary even as the corpous grows whereas the mappers may hit data limitations isssues). I also wanted practice passing one map-reduce job to another. 


In [40]:
# part a - do your work in train_mapper.py and train_reducer.py then RUN THIS CELL AS IS
!chmod a+x NaiveBayes/train_mapper.py
!chmod a+x NaiveBayes/train_mapper.py
!echo "=========== MAPPER DOCSTRING ============"
!head -n 8 NaiveBayes/train_mapper.py | tail -n 6
!echo "=========== REDUCER DOCSTRING ============"
!head -n 8 NaiveBayes/train_reducer.py | tail -n 6

=========== MAPPER DOCSTRING ============
Mapper reads in text documents and emits word counts by class.
INPUT:                                                    
    DocID \t true_class \t subject \t body                
OUTPUT:                                                   
    partitionKey \t word \t class0_partialCount,class1_partialCount       
    
=========== REDUCER DOCSTRING ============
Reducer aggregates word counts by class and emits frequencies.

INPUT:
    <specify record format here>
OUTPUT:
    <specify record format here>


In [89]:
!cat NaiveBayes/train_mapper.py

#!/usr/bin/env python
"""
Mapper reads in text documents and emits word counts by class.
INPUT:                                                    
    DocID \t true_class \t subject \t body                
OUTPUT:                                                   
    partitionKey \t word \t class0_partialCount,class1_partialCount       
    

Instructions:
    You know what this script should do, go for it!
    (As a favor to the graders, please comment your code clearly!)
    
    A few reminders:
    1) To make sure your results match ours please be sure
       to use the same tokenizing that we have provided in
       all the other jobs:
         words = re.findall(r'[a-z]+', text-to-tokenize.lower())
         
    2) Don't forget to handle the various "totals" that you need
       for your conditional probabilities and class priors.
       
Partitioning:
    In order to send the totals to each reducer, we need to implement
    a custom partitioning strategy.
    
    We will gene

In [42]:
!cat NaiveBayes/train_reducer.py

#!/usr/bin/env python
"""
Reducer aggregates word counts by class and emits frequencies.

INPUT:
    <specify record format here>
OUTPUT:
    <specify record format here>
    
Instructions:
    Again, you are free to design a solution however you see 
    fit as long as your final model meets our required format
    for the inference job we designed in Question 8. Please
    comment your code clearly and concisely.
    
    A few reminders: 
    1) Don't forget to emit Class Priors (with the right key).
    2) In python2: 3/4 = 0 and 3/float(4) = 0.75
"""
##################### YOUR CODE HERE ####################

import re                                                   
import sys                                                  
import numpy as np      

from operator import itemgetter
import os

#initialize
current_word = None

grand_total = 0
ham_line_count, spam_line_count = 0, 0
ham_partial_count, spam_partial_count = 0, 0
class0_partialCount, class1_partialCount = 0, 0
unique_

In [43]:
!cat NaiveBayes/train_reducer_smooth.py

#!/usr/bin/env python

import os
import sys                                                  
import numpy as np  

#################### YOUR CODE HERE ###################

# Mapper format
#A  !line_counter  1  3
#A  !part_counter  3  8
#A  !total         0  11
#A  beijing        0  1
#A  chinese        0  1
#A  chinese        0  1


# Loop over the data to get the unique word count
#initialize
current_word = None
unique_word_counter = 0
total_counter = 0
line_counter_ham, line_counter_spam = 0,0
part_counter_ham, part_counter_spam = 0,0


for line in sys.stdin:
    key, word, col_3, col_4 = line.split('\t')
    col_3 = int(col_3)
    col_4 = int(col_4)
    if word != current_word and word != "!total" and word != "!line_counter" and word != "!part_counter":
        unique_word_counter += 1
    current_word = word
    
    if word == "!total":
        total_counter += col_4
    elif word == "!line_counter":
        line_counter_ham += col_3   
        line_counter_spam += col_4   
    e

__`part b starts here`:__ MNB _without_ Smoothing (training on Chinese Example Corpus).

In [44]:
# part b - write a unit test for your mapper here
!cat NaiveBayes/chineseTrain.txt | NaiveBayes/train_mapper.py | sort -k2,2 | column -t

A  !line_counter  1  3
A  !part_counter  3  8
A  !total         0  11
A  beijing        0  1
A  chinese        0  1
A  chinese        0  1
A  chinese        0  1
A  chinese        0  1
A  chinese        0  1
A  chinese        1  0
A  japan          1  0
A  macao          0  1
A  shanghai       0  1
A  tokyo          1  0


In [45]:
# part b - write a unit test for your reducer here
!echo -e '\ntest:\n'
!echo -e 'A\t!total\t0\t3\nA\t!part_counter\t1\t2\nA\t!line_counter\t1\t2\nA\tchinese\t0\t1\nA\tjapan\t1\t0\nA\tchinese\t0\t1' | sort -k2,2
!echo -e '\nreducer results:\n'
!echo -e 'A\t!total\t0\t3\nA\t!part_counter\t1\t2\nA\t!line_counter\t1\t2\nA\tchinese\t0\t1\nA\tjapan\t1\t0\nA\tchinese\t0\t1' | sort -k2,2 | NaiveBayes/train_reducer.py | column -t


test:

A	!line_counter	1	2
A	!part_counter	1	2
A	!total	0	3
A	chinese	0	1
A	chinese	0	1
A	japan	1	0

reducer results:

chinese      0,2,0.0,1.0
japan        1,0,1.0,0.0
ClassPriors  1,2,0.3333333333333333,0.6666666666666666


In [46]:
# part b - write a systems test for your mapper + reducer together here
!cat NaiveBayes/chineseTrain.txt | NaiveBayes/train_mapper.py | sort -k2,2 | NaiveBayes/train_reducer.py | column -t

beijing      0,1,0.0,0.125
chinese      1,5,0.3333333333333333,0.625
japan        1,0,0.3333333333333333,0.0
macao        0,1,0.0,0.125
shanghai     0,1,0.0,0.125
tokyo        1,0,0.3333333333333333,0.0
ClassPriors  1,3,0.25,0.75


In [47]:
# part b - clear (and name) an output directory in HDFS for your unsmoothed chinese NB model
!hdfs dfs -rm -r {HDFS_DIR}/unsmoothed-model-output

rm: `/user/root/HW2//unsmoothed-model-output': No such file or directory


In [48]:
!hdfs dfs -rm -r {HDFS_DIR}/unsmoothed-model-output
# part b - write your hadoop streaming job
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=3 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options=-k2,2 \
  -D mapreduce.partition.keypartitioner.options=-k1,1  \
  -files NaiveBayes/train_mapper.py,NaiveBayes/train_reducer.py \
  -mapper train_mapper.py \
  -reducer train_reducer.py \
  -input {HDFS_DIR}/chineseTrain.txt \
  -output {HDFS_DIR}/unsmoothed-model-output \
  -cmdenv PATH={PATH} \
  -numReduceTasks 1 \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner


rm: `/user/root/HW2//unsmoothed-model-output': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob4939103661031575368.jar tmpDir=null
21/09/22 03:49:06 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/22 03:49:06 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/22 03:49:07 INFO mapred.FileInputFormat: Total input paths to process : 1
21/09/22 03:49:07 INFO mapreduce.JobSubmitter: number of splits:2
21/09/22 03:49:07 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1632272621573_0006
21/09/22 03:49:08 INFO impl.YarnClientImpl: Submitted application application_1632272621573_0006
21/09/22 03:49:08 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1632272621573_0006/
21/09/22 03:49:08 INFO mapreduce.Job: Running job: job_1632272621573_0006
21/09/22 03:49:18 INFO mapreduce.Job: Job job_1632272621573_0006 running in uber m

In [49]:
# part b - extract your results (i.e. model) to a local file
!hdfs dfs -cat {HDFS_DIR}/unsmoothed-model-output/part-000* > NaiveBayes/unsmoothedResults.txt

In [50]:
# part b - print your model so that we can confirm that it matches expected results
!cat NaiveBayes/unsmoothedResults.txt

beijing	0,1,0.0,0.125
chinese	1,5,0.3333333333333333,0.625
japan	1,0,0.3333333333333333,0.0
macao	0,1,0.0,0.125
shanghai	0,1,0.0,0.125
tokyo	1,0,0.3333333333333333,0.0
ClassPriors	1,3,0.25,0.75


In [51]:
!cat NaiveBayes/chineseTrain.txt

D1	1		Chinese Beijing Chinese
D2	1		Chinese Chinese Shanghai
D3	1		Chinese Macao
D4	0		Tokyo Japan Chinese


__`part e starts here`:__ MNB _with_ Smoothing (training on Chinese Example Corpus).

In [52]:
# part e - write a unit test for your NEW reducer here
!chmod a+x NaiveBayes/train_reducer_smooth.py

!echo -e '\ntest:\n'
!echo -e 'A\t!total\t0\t3\nA\t!part_counter\t1\t2\nA\t!line_counter\t1\t2\nA\tchinese\t0\t1\nA\tjapan\t1\t0\nA\tchinese\t0\t1' | sort -k2,2
!echo -e '\nsmoothed values:\n'
!echo -e 'A\t!total\t0\t3\nA\t!part_counter\t1\t2\nA\t!line_counter\t1\t2\nA\tchinese\t0\t1\nA\tjapan\t1\t0\nA\tchinese\t0\t1' | sort -k2,2 | NaiveBayes/train_reducer_smooth.py | sort -k2,2 | column -t
!echo -e '\nsmoothed reducer values:\n'
!echo -e 'A\t!total\t0\t3\nA\t!part_counter\t1\t2\nA\t!line_counter\t1\t2\nA\tchinese\t0\t1\nA\tjapan\t1\t0\nA\tchinese\t0\t1' | sort -k2,2 | NaiveBayes/train_reducer_smooth.py | sort -k2,2 | NaiveBayes/train_reducer.py | column -t




test:

A	!line_counter	1	2
A	!part_counter	1	2
A	!total	0	3
A	chinese	0	1
A	chinese	0	1
A	japan	1	0

smoothed values:

A  !!unique_words  0  2
A  !line_counter   1  2
A  !part_counter   1  2
A  !total          0  3
A  chinese         0  1
A  chinese         0  1
A  japan           1  0

smoothed reducer values:

chinese      0,2,0.3333333333333333,0.75
japan        2,1,0.6666666666666666,0.25
ClassPriors  1,2,0.3333333333333333,0.6666666666666666


In [53]:
!chmod a+x NaiveBayes/train_mapper.py
!chmod a+x NaiveBayes/train_reducer.py
!chmod a+x NaiveBayes/train_reducer_smooth.py

# part e - write a systems test for your mapper + reducer together here
!cat NaiveBayes/chineseTrain.txt | NaiveBayes/train_mapper.py | sort -k2,2 
!cat NaiveBayes/chineseTrain.txt | NaiveBayes/train_mapper.py | sort -k2,2 | NaiveBayes/train_reducer_smooth.py | sort -k2,2 | NaiveBayes/train_reducer.py | column -t



A	!line_counter	1	3
A	!part_counter	3	8
A	!total	0	11
A	beijing	0	1
A	chinese	0	1
A	chinese	0	1
A	chinese	0	1
A	chinese	0	1
A	chinese	0	1
A	chinese	1	0
A	japan	1	0
A	macao	0	1
A	shanghai	0	1
A	tokyo	1	0
beijing      1,2,0.1111111111111111,0.14285714285714285
chinese      1,5,0.2222222222222222,0.42857142857142855
japan        2,1,0.2222222222222222,0.07142857142857142
macao        1,2,0.1111111111111111,0.14285714285714285
shanghai     0,1,0.1111111111111111,0.14285714285714285
tokyo        2,1,0.2222222222222222,0.07142857142857142
ClassPriors  1,3,0.25,0.75


In [54]:
# part e - clear (and name) an output directory in HDFS for your SMOOTHED chinese NB model
!hdfs dfs -rm -r {HDFS_DIR}/smoothed-model-output
!hdfs dfs -rm -r {HDFS_DIR}/smoothed-model-output-tmp

rm: `/user/root/HW2//smoothed-model-output': No such file or directory
rm: `/user/root/HW2//smoothed-model-output-tmp': No such file or directory


In [55]:
# part e - write your hadoop streaming job
!hdfs dfs -rm -r {HDFS_DIR}/smoothed-model-output
!hdfs dfs -rm -r {HDFS_DIR}/smoothed-model-output-tmp

!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=3 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options=-k2,2 \
  -D mapreduce.partition.keypartitioner.options=-k1,1 \
  -files NaiveBayes/train_mapper.py,NaiveBayes/train_reducer_smooth.py \
  -mapper train_mapper.py \
  -reducer train_reducer_smooth.py \
  -input {HDFS_DIR}/chineseTrain.txt \
  -output {HDFS_DIR}/smoothed-model-output-tmp \
  -cmdenv PATH={PATH} \
  -numReduceTasks 1 \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner

!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=3 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options=-k2,2 \
  -D mapreduce.partition.keypartitioner.options=-k1,1 \
  -files NaiveBayes/train_reducer.py \
  -mapper /bin/cat \
  -reducer train_reducer.py \
  -input {HDFS_DIR}/smoothed-model-output-tmp/* \
  -output {HDFS_DIR}/smoothed-model-output \
  -cmdenv PATH={PATH} \
  -numReduceTasks 1 \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner

rm: `/user/root/HW2//smoothed-model-output': No such file or directory
rm: `/user/root/HW2//smoothed-model-output-tmp': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob49884312148421289.jar tmpDir=null
21/09/22 03:50:06 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/22 03:50:06 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/22 03:50:07 INFO mapred.FileInputFormat: Total input paths to process : 1
21/09/22 03:50:07 INFO mapreduce.JobSubmitter: number of splits:2
21/09/22 03:50:07 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1632272621573_0007
21/09/22 03:50:08 INFO impl.YarnClientImpl: Submitted application application_1632272621573_0007
21/09/22 03:50:08 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1632272621573_0007/
21/09/22 03:50:08 INFO mapreduce.Job: Running job: job_1632272621573_0007
21/09/22 03

In [57]:
# part e - extract your results (i.e. model) to a local file
!hdfs dfs -cat {HDFS_DIR}/smoothed-model-output-tmp/part-000* > NaiveBayes/smoothedResults_tmp.txt
!hdfs dfs -cat {HDFS_DIR}/smoothed-model-output/part-000* > NaiveBayes/smoothedResults.txt

In [58]:
# part e - print your model (above local file) so that we can confirm that it matches expected results
!cat NaiveBayes/smoothedResults.txt

beijing	1,2,0.1111111111111111,0.14285714285714285
chinese	1,5,0.2222222222222222,0.42857142857142855
japan	2,1,0.2222222222222222,0.07142857142857142
macao	1,2,0.1111111111111111,0.14285714285714285
shanghai	0,1,0.1111111111111111,0.14285714285714285
tokyo	2,1,0.2222222222222222,0.07142857142857142
ClassPriors	1,3,0.25,0.75


# Question 9: Enron Ham/Spam NB Classifier & Results.

Fantastic work. We're finally ready to perform Spam Classification on the Enron Corpus. In this question you'll run the analysis you've developed, report its performance.

### Q9 Tasks:
* __a) train/test split:__ Run the provided code to split our Enron file into a training set and testing set then load them into HDFS. [`NOTE:` _Make sure you re calculate the vocab size for just the training set!_]

* __b) train 2 models:__ Write Hadoop Streaming jobs to train MNB Models on the training set with and without smoothing. Save your models to local files at __`NaiveBayes/Unsmoothed/NBmodel.txt`__ and __`NaiveBayes/Smoothed/NBmodel.txt`__. [`NOTE:` _This naming is important because we wrote our classification task so that it expects a file of that name... if this inelegance frustrates you there is an alternative that would involve a few adjustments to your code [read more about it here](http://www.tnoda.com/blog/2013-11-23)._] Finally run the checks that we provide to confirm that your results are correct.


* __c) code:__ Recall that we designed our classification job with just a mapper. An efficient way to report the performance of our models would be to simply add a reducer phase to this job and compute precision and recall right there. Complete the code in __`NaiveBayes/evaluation_reducer.py`__ and then write Hadoop jobs to evaluate your two models on the test set. Report their performance side by side. [`NOTE:` if you need a refresher on precision, recall and F1-score [Wikipedia](https://en.wikipedia.org/wiki/F1_score) is a good resource.]


* __d) short response:__ Compare the performance of your two models. What do you notice about the unsmoothed model's predictions? Can you guess why this is happening? Which evaluation measure do you think is most relevant in our use case? [`NOTE:` _Feel free to answer using your common sense but if you want more information on evaluating the classification task checkout_ [this blogpost](https://tryolabs.com/blog/2013/03/25/why-accuracy-alone-bad-measure-classification-tasks-and-what-we-can-do-about-it/
) or [this paper](http://www.flinders.edu.au/science_engineering/fms/School-CSEM/publications/tech_reps-research_artfcts/TRRA_2007.pdf
)]



### Q9 Student Answers:
> __d)__  The unsmoothed model's precision score is 100% even though all of the other performance metrics are pretty poor whereas the accuracy, recall, and F1 score are much better for the smoothed model but the precision is worst. That is because the smoothed model is predicting more false positives than the regular model - likely due to the behavior of the class priors and the additive properties of increasing the counts of rare words.   


__Test/Train split__

In [59]:
# part a - test/train split (RUN THIS CELL AS IS)
!head -n 80 data/enronemail_1h.txt > data/enron_train.txt
!tail -n 20 data/enronemail_1h.txt > data/enron_test.txt
!hdfs dfs -copyFromLocal data/enron_train.txt {HDFS_DIR}
!hdfs dfs -copyFromLocal data/enron_test.txt {HDFS_DIR}

__Training__ (Enron MNB Model _without smoothing_ )

In [61]:
# part b -  Unsmoothed model (FILL IN THE MISSING CODE BELOW)

# clear the output directory
!hdfs dfs -rm -r {HDFS_DIR}/enron-model

# hadoop command
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=3 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options=-k2,2 \
  -D mapreduce.partition.keypartitioner.options=-k1,1  \
  -files NaiveBayes/train_mapper.py,NaiveBayes/train_reducer.py \
  -mapper train_mapper.py \
  -reducer train_reducer.py \
  -input {HDFS_DIR}/enron_train.txt \
  -output {HDFS_DIR}/enron-model \
  -cmdenv PATH={PATH} \
  -numReduceTasks 1 \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner



# save the model locally
!mkdir NaiveBayes/Unsmoothed
!hdfs dfs -cat {HDFS_DIR}/enron-model/part-000* > NaiveBayes/Unsmoothed/NBmodel.txt

rm: `/user/root/HW2//enron-model': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob4681067299969289088.jar tmpDir=null
21/09/22 03:52:20 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/22 03:52:20 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/22 03:52:21 INFO mapred.FileInputFormat: Total input paths to process : 1
21/09/22 03:52:21 INFO mapreduce.JobSubmitter: number of splits:2
21/09/22 03:52:21 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1632272621573_0010
21/09/22 03:52:22 INFO impl.YarnClientImpl: Submitted application application_1632272621573_0010
21/09/22 03:52:22 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1632272621573_0010/
21/09/22 03:52:22 INFO mapreduce.Job: Running job: job_1632272621573_0010
21/09/22 03:52:32 INFO mapreduce.Job: Job job_1632272621573_0010 running in uber mode : false


In [62]:
# part b - check your UNSMOOTHED model results (RUN THIS CELL AS IS)
!grep assistance NaiveBayes/Unsmoothed/NBmodel.txt
# EXPECTED OUTPUT: assistance	2,4,0.000172547666293,0.000296823983378

assistance	2,4,0.0001725476662928134,0.00029682398337785694


In [63]:
# part b - check your UNSMOOTHED model results (RUN THIS CELL AS IS)
!grep money NaiveBayes/Unsmoothed/NBmodel.txt
# EXPECTED OUTPUT: money	1,22,8.62738331464e-05,0.00163253190858

money	1,22,8.62738331464067e-05,0.001632531908578213


__Training__ (Enron MNB Model _with Laplace +1 smoothing_ )

In [64]:
# part b -  Smoothed model (FILL IN THE MISSING CODE BELOW)

# clear the output directory
!hdfs dfs -rm -r {HDFS_DIR}/smooth-model
!hdfs dfs -rm -r {HDFS_DIR}/smooth-model-tmp

# hadoop command
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=3 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options=-k2,2 \
  -D mapreduce.partition.keypartitioner.options=-k1,1 \
  -files NaiveBayes/train_mapper.py,NaiveBayes/train_reducer_smooth.py \
  -mapper train_mapper.py \
  -reducer train_reducer_smooth.py \
  -input {HDFS_DIR}/enron_train.txt \
  -output {HDFS_DIR}/smooth-model-tmp \
  -cmdenv PATH={PATH} \
  -numReduceTasks 1 \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner

!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=3 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options=-k2,2 \
  -D mapreduce.partition.keypartitioner.options=-k1,1 \
  -files NaiveBayes/train_reducer.py \
  -mapper /bin/cat \
  -reducer train_reducer.py \
  -input {HDFS_DIR}/smooth-model-tmp/* \
  -output {HDFS_DIR}/smooth-model \
  -cmdenv PATH={PATH} \
  -numReduceTasks 1 \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner


# save the model locally
!mkdir NaiveBayes/Smoothed
!hdfs dfs -cat {HDFS_DIR}/smooth-model/part-000* > NaiveBayes/Smoothed/NBmodel.txt

rm: `/user/root/HW2//smooth-model': No such file or directory
rm: `/user/root/HW2//smooth-model-tmp': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob7620113854052395148.jar tmpDir=null
21/09/22 03:53:05 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/22 03:53:05 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/22 03:53:06 INFO mapred.FileInputFormat: Total input paths to process : 1
21/09/22 03:53:06 INFO mapreduce.JobSubmitter: number of splits:2
21/09/22 03:53:06 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1632272621573_0011
21/09/22 03:53:07 INFO impl.YarnClientImpl: Submitted application application_1632272621573_0011
21/09/22 03:53:07 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1632272621573_0011/
21/09/22 03:53:07 INFO mapreduce.Job: Running job: job_1632272621573_0011
21/09/22 03:53:16 INFO mapr

In [65]:
# part b - check your SMOOTHED model results (RUN THIS CELL AS IS)
!grep assistance NaiveBayes/Smoothed/NBmodel.txt
# EXPECTED OUTPUT: assistance	2,4,0.000185804533631,0.000277300205202

assistance	2,4,0.0001858045336306206,0.00027730020520215184


In [66]:
# part b - check your SMOOTHED model results (RUN THIS CELL AS IS)
!grep money NaiveBayes/Smoothed/NBmodel.txt
# EXPECTED OUTPUT: money	1,22,0.000123869689087,0.00127558094393

money	2,23,0.0001238696890870804,0.0012755809439298986


__Evaluation__

In [67]:
# part c - write your code in NaiveBayes/evaluation_reducer.py then RUN THIS
!chmod a+x NaiveBayes/evaluation_reducer.py

In [68]:
!cat NaiveBayes/evaluation_reducer.py

#!/usr/bin/env python
"""
Reducer to calculate precision and recall as part
of the inference phase of Naive Bayes.
INPUT:
    ID \t true_class \t P(ham|doc) \t P(spam|doc) \t predicted_class
OUTPUT:
    precision \t ##
    recall \t ##
    accuracy \t ##
    F-score \t ##
         
Instructions:
    Complete the missing code to compute these^ four
    evaluation measures for our classification task.
    
    Note: if you have no True Positives you will not 
    be able to compute the F1 score (and maybe not 
    precision/recall). Your code should handle this 
    case appropriately feel free to interpret the 
    "output format" above as a rough suggestion. It
    may be helpful to also print the counts for true
    positives, false positives, etc.
"""
import sys
import numpy as np

# initialize counters
FP = 0.0 # false positives
FN = 0.0 # false negatives
TP = 0.0 # true positives
TN = 0.0 # true negatives
# current_docID = None
doc_num = 0
docID_list = []

# read from STDIN
for lin

In [69]:
# part c - unit test your evaluation job on the chinese model (RUN THIS CELL AS IS)
!cat NaiveBayes/chineseTest.txt | NaiveBayes/classify_mapper.py 
!cat NaiveBayes/chineseTest.txt | NaiveBayes/classify_mapper.py | NaiveBayes/evaluation_reducer.py

d5	1	-8.90668134500626	-8.10769031284611	1
d6	1	-5.780743515794329	-4.179502370564408	1
d7	0	-6.591673732011658	-7.511706880737811	0
d8	0	-4.394449154674438	-5.565796731681498	0
d5	1	-8.90668134500626	-8.10769031284611	 True
d6	1	-5.780743515794329	-4.179502370564408	 True
d7	0	-6.591673732011658	-7.511706880737811	 True
d8	0	-4.394449154674438	-5.565796731681498	 True
Documents:	4
True Positives:	2.0
True Negatives:	2.0
False Positives:	0.0
False Negatives:	0.0
Accuracy:	1.0
Precision:	1.0
Recall:	1.0
F-Score:	1.0


In [70]:
# part c - Evaluate the UNSMOOTHED Model Here (FILL IN THE MISSING CODE)

# clear output directory
!hdfs dfs -rm -r {HDFS_DIR}/enron-output-unsmooth

!hadoop jar {JAR_FILE} \
  -files NaiveBayes/classify_mapper.py,NaiveBayes/evaluation_reducer.py,NaiveBayes/Unsmoothed/NBmodel.txt\
  -mapper classify_mapper.py\
  -reducer evaluation_reducer.py\
  -input {HDFS_DIR}/enron_test.txt \
  -output {HDFS_DIR}/enron-output-unsmooth \
  -cmdenv PATH={PATH}

# retrieve results locally!
!mkdir NaiveBayes/Unsmoothed
!hdfs dfs -cat {HDFS_DIR}/enron-output-unsmooth/part-000* > NaiveBayes/Unsmoothed/Enron_classified.txt



rm: `/user/root/HW2//enron-output-unsmooth': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob999516715194631468.jar tmpDir=null
21/09/22 03:54:18 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/22 03:54:18 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/22 03:54:19 INFO mapred.FileInputFormat: Total input paths to process : 1
21/09/22 03:54:20 INFO mapreduce.JobSubmitter: number of splits:2
21/09/22 03:54:20 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1632272621573_0013
21/09/22 03:54:20 INFO impl.YarnClientImpl: Submitted application application_1632272621573_0013
21/09/22 03:54:20 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1632272621573_0013/
21/09/22 03:54:20 INFO mapreduce.Job: Running job: job_1632272621573_0013
21/09/22 03:54:29 INFO mapreduce.Job: Job job_1632272621573_0013 running in uber mode

In [71]:
# part c - Evaluate the SMOOTHED Model Here (FILL IN THE MISSING CODE)
# hadoop job

# clear output directory
!hdfs dfs -rm -r {HDFS_DIR}/enron-output-smooth

!hadoop jar {JAR_FILE} \
  -files NaiveBayes/classify_mapper.py,NaiveBayes/evaluation_reducer.py,NaiveBayes/Smoothed/NBmodel.txt\
  -mapper classify_mapper.py\
  -reducer evaluation_reducer.py\
  -input {HDFS_DIR}/enron_test.txt \
  -output {HDFS_DIR}/enron-output-smooth \
  -cmdenv PATH={PATH}

# !hadoop jar {JAR_FILE} \
#   -D stream.num.map.output.key.fields=3 \
#   -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
#   -D mapreduce.partition.keycomparator.options=-k2,2 \
#   -D mapreduce.partition.keypartitioner.options=-k1,1 \
#   -files NaiveBayes/model_sort_mapper.py,NaiveBayes/evaluation_reducer.py,NaiveBayes/Smoothed/NBmodel.txt\
#   -mapper classify_mapper.py\
#   -reducer train_reducer_smooth.py \
#   -input {HDFS_DIR}/enron_train.txt \
#   -output {HDFS_DIR}/enron-output-smooth \
#   -cmdenv PATH={PATH} \
#   -numReduceTasks 1 \
#   -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner

# retrieve results locally
!mkdir NaiveBayes/Smoothed
!hdfs dfs -cat {HDFS_DIR}/enron-output-smooth/part-000* > NaiveBayes/Smoothed/Enron_classified.txt


rm: `/user/root/HW2//enron-output-smooth': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob5305433402467397449.jar tmpDir=null
21/09/22 03:54:58 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/22 03:54:58 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/22 03:54:59 INFO mapred.FileInputFormat: Total input paths to process : 1
21/09/22 03:54:59 INFO mapreduce.JobSubmitter: number of splits:2
21/09/22 03:54:59 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1632272621573_0014
21/09/22 03:55:00 INFO impl.YarnClientImpl: Submitted application application_1632272621573_0014
21/09/22 03:55:00 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1632272621573_0014/
21/09/22 03:55:00 INFO mapreduce.Job: Running job: job_1632272621573_0014
21/09/22 03:55:09 INFO mapreduce.Job: Job job_1632272621573_0014 running in uber mode 

In [72]:
# part c - display results 
# NOTE: feel free to modify the tail commands to match the format of your results file
print('=========== UNSMOOTHED MODEL ============')
!tail -n 9 NaiveBayes/Unsmoothed/Enron_classified.txt
print('=========== SMOOTHED MODEL ============')
!tail -n 9 NaiveBayes/Smoothed/Enron_classified.txt

=========== UNSMOOTHED MODEL ============
Documents:	20
True Positives:	1.0
True Negatives:	9.0
False Positives:	0.0
False Negatives:	10.0
Accuracy:	0.5
Precision:	1.0
Recall:	0.09090909090909091
F-Score:	0.16666666666666666
=========== SMOOTHED MODEL ============
Documents:	20
True Positives:	11.0
True Negatives:	6.0
False Positives:	3.0
False Negatives:	3.0
Accuracy:	0.7391304347826086
Precision:	0.7857142857142857
Recall:	0.7857142857142857
F-Score:	0.7857142857142857


__`EXPECTED RESULTS:`__ 
<table>
<th>Unsmoothed Model</th>
<th>Smoothed Model</th>
<tr>
<td><pre>
# Documents:	20
True Positives:	1
True Negatives:	9
False Positives:	0
False Negatives:	10
Accuracy	0.5
Precision	1.0
Recall	0.0909
F-Score	0.1666
</pre></td>
<td><pre>
# Documents:	20
True Positives:	11
True Negatives:	6
False Positives:	3
False Negatives:	0
Accuracy	0.85
Precision	0.7857
Recall	1.0
F-Score	0.88
</pre></td>
</tr>
</table>

__`NOTE:`__ _Don't be too disappointed if these seem low to you. We've trained and tested on a very very small corpus... bigger datasets coming soon!_

# Question 10: Custom Partitioning and Secondary Sort

Now that we have our model, we can analyse the results and think about future improvements.

### Q10 Tasks:

* __a) code + short response:__ Let's look at the top ten words with the highest conditional probability in `Spam` and in `Ham`. We'll do this by writing a Hadoop job that sorts the model file (`NaiveBayes/Smoothed/NBmodel.py`). Normally we'd have to run two jobs -- one that sorts on $P(word|ham)$ and another that sorts on $P(word|spam)$. However if we slighly modify the data format in the model file then we can get the top words in each class with just one job. We've written a mapper that will do just this for you. Read through __`NaiveBayes/model_sort_mapper.py`__ and then briefly explain how this mapper will allow us to partition and sort our model file. Write a Hadoop job that uses our mapper and `/bin/cat` for a reducer to partition and sort. Print out the top 10 words in each class (where 'top' == highest conditional probability).[`HINT:` _this should remind you a lot of what we did in Question 6._]


* __b) short response:__ What do you notice about the 'top words' we printed in `a`? How would increasing the smoothing parameter 'k' affect the probabilities for the top words that you identified for 'a'. How would they affect the probabilities of words that occur much more in one class than another? In summary, how does the smoothing parameter 'k' affect the bias and the variance of our model. [`NOTE:` _you do not need to code anything for this task, but if you are struggling with it you could try changing 'k' and see what happens to the test set. We don't recommend doing this exploration with the Enron data because it will be harder to see the impact with such a big vocabulary_]

### Q10 Student Answers:
> __a)__ The mapper prints a class typer per word in eahc trained NB and strips the word, payload, and class probability for printing. It then partitions on the class type and sorts on the probability such that we can easily see the max probability per class. 

> __b)__ Increasing the smoothing paramter decreases the delta's between probabilities in words. As k increases indefinetly, the conditional probabilities for each word approach the class priors values. For words that show up much more in once class then another, it lessens the impact of this difference as we smooth it out with a signficantly high K value. It would decrease the variance of the model as it relies more heavily on the class priors to make a prediction but increases the bias. 




In [83]:
# part a - write your Hadoop job here (sort smoothed model on P(word|class))

# clear output directory
!hdfs dfs -rm -r {HDFS_DIR}/enron-output-smooth-top10

# hadoop job
!hdfs dfs -copyFromLocal NaiveBayes/Smoothed/NBmodel.txt {HDFS_DIR}

!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=4 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options=-k4,4 \
  -D mapreduce.partition.keypartitioner.options=-k3,3 \
  -files NaiveBayes/model_sort_mapper.py\
  -mapper model_sort_mapper.py\
  -reducer /bin/cat \
  -input {HDFS_DIR}/NBmodel.txt \
  -output {HDFS_DIR}/enron-output-smooth-top10 \
  -cmdenv PATH={PATH} \
  -numReduceTasks 2 \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner


Deleted /user/root/HW2/enron-output-smooth-top10
copyFromLocal: `/user/root/HW2/NBmodel.txt': File exists
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob2076727111707985577.jar tmpDir=null
21/09/22 04:30:34 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/22 04:30:34 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/09/22 04:30:35 INFO mapred.FileInputFormat: Total input paths to process : 1
21/09/22 04:30:35 INFO mapreduce.JobSubmitter: number of splits:2
21/09/22 04:30:35 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1632272621573_0019
21/09/22 04:30:36 INFO impl.YarnClientImpl: Submitted application application_1632272621573_0019
21/09/22 04:30:36 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1632272621573_0019/
21/09/22 04:30:36 INFO mapreduce.Job: Running job: job_1632272621573_0019
21/09/22 04:30:44 INFO mapreduce.Job: Job job_163

In [87]:
# part b - print top words in each class
!hdfs dfs -ls {HDFS_DIR}/enron-output-smooth-top10/

!mkdir NaiveBayes/Smoothed
# !hdfs dfs -cat {HDFS_DIR}/enron-output-smooth-top10/part-000* | head
# !hdfs dfs -cat {HDFS_DIR}/enron-output-smooth-top10/part-000* | tail

for indx in range(2):
    print(f"\n================ Part 0000{indx}================\n")
    !hdfs dfs -cat {HDFS_DIR}/enron-output-smooth-top10/part-0000{indx} | head -n 10
#     break

Found 3 items
-rw-r--r--   1 root supergroup          0 2021-09-22 04:31 /user/root/HW2/enron-output-smooth-top10/_SUCCESS
-rw-r--r--   1 root supergroup     352100 2021-09-22 04:30 /user/root/HW2/enron-output-smooth-top10/part-00000
-rw-r--r--   1 root supergroup     357290 2021-09-22 04:31 /user/root/HW2/enron-output-smooth-top10/part-00001
mkdir: cannot create directory `NaiveBayes/Smoothed': File exists

================ Part 00000================

generalist	2,1,0.0001238696890870804,5.546004104043037e-05	ham	0.00012386968909	
lamb	2,1,0.0001238696890870804,5.546004104043037e-05	ham	0.00012386968909	
absent	2,1,0.0001238696890870804,5.546004104043037e-05	ham	0.00012386968909	
lagrasta	1,0,0.0001238696890870804,5.546004104043037e-05	ham	0.00012386968909	
accepts	2,1,0.0001238696890870804,5.546004104043037e-05	ham	0.00012386968909	
lacrecia	2,1,0.0001238696890870804,5.546004104043037e-05	ham	0.00012386968909	
accomodates	1,0,0.0001238696890870804,5.546004104043037e-05	ham	0.00012386

### Congratulations, you have completed HW2! Please refer to the readme for submission instructions.

If you would like to provide feedback regarding this homework, please use the survey at: https://docs.google.com/forms/d/e/1FAIpQLSce9feiQeSkdP43A0ZYui1tMGIBfLfzb0rmgToQeZD9bXXX8Q/viewform